<a href="https://colab.research.google.com/github/GeertD/blog/blob/main/BGT_BioMethane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Membrane Module
For testing Python code for a membrane module.
Once this runs, plug into DWSim and use Python.NET.

## Imports

In [ ]:
!pip install gekko
from gekko import GEKKO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 38.4 MB/s eta 0:00:00


# Gekko basic NLP

Solve a set of Non Linear equations.

Note: no objective function, no optimization problem.

In [ ]:
from gekko import GEKKO

m = GEKKO(remote=False)             # create GEKKO model
x = m.Var(value=0)
y = m.Var(value=1)
z = m.Var(value=0)

eq1 = x + 2*y == 0
eq2 = x**2 + y**2 == 1
eq3 = x + y**3 - z == 5

m.Equations([eq1, eq2, eq3])

result = m.solve(disp=True, debug=False)

print([x.value[0],y.value[0], z.value[0]])
print(result)


 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            3
   Intermediates:            0
   Connections  :            0
   Equations    :            3
   Residuals    :            3
 
 Number of state variables:              3
 Number of total equations: -            3
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  4.00000E-20  1.00000E+00
    1  1.21600E-20  5.00000E-01
    2 

Non Linear Programming optimization example.

[Source](https://apmonitor.com/wiki/index.php/Main/GekkoPythonOptimization).

In [ ]:
from gekko import GEKKO
import numpy as np

#Initialize Model
m = GEKKO(remote=False)

#define parameter
eq = m.Param(value=40)

#initialize variables
x1,x2,x3,x4 = [m.Var(value=2, lb=1.0, ub=5.0) for i in range(4)]

#Equations
m.Equation(x1*x2*x3*x4>=25)
m.Equation(x1**2+x2**2+x3**2+x4**2==eq)

#Objective
m.Obj(x1*x4*(x1+x2+x3)+x3)

#Set global options
m.options.IMODE = 3 #steady state optimization

#Solve simulation
with redirect_stdout(io.StringIO()) as f:
    m.solve(disp=True)
result = f.getvalue()

#Results
print('Results:')
print('x1: ' + str(x1.value))
print('x2: ' + str(x2.value))
print('x3: ' + str(x3.value))
print('x4: ' + str(x4.value))

print(result)

NameError: name 'redirect_stdout' is not defined

In [ ]:
from contextlib import redirect_stdout
import io
with redirect_stdout(io.StringIO()) as f:
    help(pow)
s = f.getvalue()

In [ ]:
s

# Basic model
Basic model for gas permeation.

## Counter-Current model

In [ ]:
from gekko import GEKKO

In [223]:
# Constants
GPU_2_SI = 3.35E-10 # convert GPU to mol/m2.s.Pa

# Parameters
# Data from Pan(1986), fig 6
# all in SI units
length_fiber = 0.36   # m
OD_fiber = 200e-6   # m
number_fibers = 20
A = OD_fiber*3.1415*length_fiber*number_fibers  # membrane area (m2)

components = ["H2S", "CO2", "CH4", "N2"]
# q_GPU = [40.05, 1.11, 0.31, 0.06]   # permeances in GPU
# q = [q_el*GPU_2_SI for q_el in q_GPU]   # CHECK, convert to numpy arrays
q = [2.45e-8, 2.45e-8, 0.1324e-8, 0.134e-8]
flow_mode = "counter-current"
R = len(components) # number of components
N = 30   # number of sections
Ak = A/N  # membrane area per section (m2)

Q_feed = 10e-3  # mol/s (max cut: 0.095e-3)
x_feed = [0.2209, 0.0338, 0.6605, 0.0837]   # mole fractions

Q_sweep = 0.0  # mol/s
x_sweep = [0.0, 0.0, 0.0, 0.0]   # mole fractions

p_feed = 5245e3  # Pa
p_permeate = 92.8e3 # Pa

# Create gekko model
m = GEKKO(remote=True)    # change to False when running locally

# Variables
# Assign two extra sections for feed (i=0) en retentate side (i=N+1)
Q_R = m.Array(m.Var, dim=(R, N+2), value=Q_feed/2, lb=0.0, ub=Q_feed)   # component flowrate retentate
Q_P = m.Array(m.Var, dim=(R, N+2), value=Q_feed/2, lb=0.0, ub=Q_feed)   # component flowrate permeate

# Fix feed and sweep side

for i,x in enumerate(x_feed):
  m.fix(Q_R[i,0], val=x*Q_feed)    # feed in
  m.fix(Q_P[i,0], val=0.0)  # not used

for i,x in enumerate(x_sweep):
  m.fix(Q_R[i,N+1], val=0.0)  # not used
  m.fix(Q_P[i,N+1], val=x*Q_sweep)  # sweep in

# Parameters
p_R = p_feed
p_P = p_permeate

# Intermediates
# Note: start and end section not used, but easier to follow indices like this
Q_Rtot = [m.Intermediate(m.sum(Q_R[:, i])) for i in range(0,N+2)]   # total flowrate retentate
Q_Ptot = [m.Intermediate(m.sum(Q_P[:, i])) for i in range(0,N+2)]   # total flowrate permeate

# Constraints
# Mass balances on retentate side
m.Equations([Q_R[j,k-1] == Q_R[j,k] + q[j]*Ak*(p_R*Q_R[j,k]/Q_Rtot[k] - p_P*Q_P[j,k]/Q_Ptot[k]) for j in range(R) for k in range(1,N+1)])

# Mass balances on permeate side - COUNTER-CURRENT
m.Equations([Q_R[j,k-1] + Q_P[j,k+1] - (Q_R[j,k] + Q_P[j,k]) == 0 for j in range(R) for k in range(1,N+1)])
print("")



In [224]:
# Check model
m.options.RTOL = 1e-8
m.options.SOLVER = 1  # 1=APOPT, 3=IPOPT
m.solve(disp=True, debug=2)
m.cleanup()
print(f"Solve status: {m.options.SOLVESTATUS}")
print(f"Solve time (ms): {m.options.SOLVETIME*1000:.2} ")

apm 34.169.61.19_gk_model65 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          256
   Intermediates:           64
   Connections  :           32
   Equations    :          304
   Residuals    :          240
 
 Number of state variables:            240
 Number of total equations: -          240
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  8.81000E-25  5.00000E-03
    1  1.31430E-26  2.25035E-06
    2  9.27190E-31  2.51335E-08
  

In [225]:
def values(gekko_array):
  # Return values from a gekko array as a list of values
  return [el[0] for el in gekko_array]

In [226]:
# Solution summary
Q_f = sum(values(Q_R[:,0]))
Q_ret = sum(values(Q_R[:,N]))
Q_perm = sum(values(Q_P[:,1]))
Q_sw = sum(values(Q_P[:,N+1]))

print("FLOWS:")
print(f"Feed flow (mol/s): {Q_f}")
print(f"Retentate flow (mol/s): {Q_ret}")
print(f"Permeate flow (mol/s): {Q_perm}")
print(f"Sweep flow (mol/s): {Q_sw}")
print(f"Relative flow balance (-): {((Q_f + Q_sw)  - (Q_ret + Q_perm))} ")
print(f"Stage cut (-): {Q_perm/Q_f}")
print("")
print("COMPOSITION:")
print(f"Components: {components}")
print(f"Retentate mole fractions: {[Q[0]/Q_ret for Q in Q_R[:,N]]}")
print(f"Permeate mole fractions: {[Q[0]/Q_perm for Q in Q_P[:,1]]}")

FLOWS:
Feed flow (mol/s): 0.009989
Retentate flow (mol/s): 0.00982895195718
Permeate flow (mol/s): 0.0001600480428484
Sweep flow (mol/s): 0.0
Relative flow balance (-): -2.839915802521631e-14 
Stage cut (-): 0.016022428956692363

COMPOSITION:
Components: ['H2S', 'CO2', 'CH4', 'N2']
Retentate mole fractions: [0.21269645167741813, 0.032544776761913925, 0.6698740957717577, 0.08488467578891035]
Permeate mole fractions: [0.7398828056408427, 0.1132094615312638, 0.13020899835519817, 0.016698734472695355]


In [202]:
Q_perm = sum(values(Q_P[:,1]))
Q_perm

0.0001600480428484

## General model
Valid for different flow modes.

In [227]:
from gekko import GEKKO

In [350]:
def membrane_model(params, disp=False):
  # Create membrane model and return results
  # Parameters: see params
  # disp: display detailed solver results
  # Returns: results

  # Constants
  GPU_2_SI = 3.35E-10 # convert GPU to mol/m2.s.Pa

  # Parameters
  if params['type_membrane'] == 'HF':
    A = params['OD_fiber']*3.1415*params['length_fiber']*params['number_fibers']
  if params['type_membrane'] == 'SW':
    A = params['length_membrane']*params['height_membrane'] # area of membrane (m2)
  q = [q_el*GPU_2_SI for q_el in params['q_GPU']]   # permeances
  R = len(params['components']) # number of components
  N = params['section_count']
  Ak = A/N  # membrane area per section (m2)
  Q_feed = params['Q_feed']
  x_feed = params['x_feed']
  Q_sweep = params['Q_sweep']
  x_sweep = params['x_sweep']
  p_R = params['p_feed']
  p_P = params['p_permeate']

  # Check data integrity
  if (len(q) != R or len(x_feed) != R or len(x_sweep) != R):
    print("Error: lists of component data (q_GPU, x_feed, x_sweep) not same length as components.")
    return None

  # Create gekko model
  m = GEKKO(remote=True)    # change to False when running locally

  # Variables
  # Assign two extra sections for feed (i=0) en retentate side (i=N+1)
  Q_R = m.Array(m.Var, dim=(R, N+2), value=Q_feed/2, lb=0.0, ub=Q_feed)   # component flowrate retentate
  Q_P = m.Array(m.Var, dim=(R, N+2), value=Q_feed/2, lb=0.0, ub=Q_feed)   # component flowrate permeate

  # Fix feed and sweep side
  for i,x in enumerate(x_feed):
    m.fix(Q_R[i,0], val=x*Q_feed)    # feed in
    m.fix(Q_P[i,0], val=0.0)  # not used

  for i,x in enumerate(x_sweep):
    m.fix(Q_R[i,N+1], val=0.0)  # not used
    m.fix(Q_P[i,N+1], val=x*Q_sweep)  # sweep in

  # Intermediates
  # Note: start and end section not used, but easier to follow indices like this
  Q_Rtot = [m.Intermediate(m.sum(Q_R[:, i])) for i in range(0,N+2)]   # total flowrate retentate
  Q_Ptot = [m.Intermediate(m.sum(Q_P[:, i])) for i in range(0,N+2)]   # total flowrate permeate

  # Constraints
  # Mass balances on retentate side
  m.Equations([Q_R[j,k-1] == Q_R[j,k] + q[j]*Ak*(p_R*Q_R[j,k]/Q_Rtot[k] - p_P*Q_P[j,k]/Q_Ptot[k]) for j in range(R) for k in range(1,N+1)])

  # Mass balances on permeate side
  if params['flow_mode'] == 'counter-current':
    m.Equations([Q_R[j,k-1] + Q_P[j,k+1] - (Q_R[j,k] + Q_P[j,k]) == 0 for j in range(R) for k in range(1,N+1)])
  elif params['flow_mode'] == 'cross-current':
    m.Equations([Q_R[j,k-1] + 0 - (Q_R[j,k] + Q_P[j,k]) == 0 for j in range(R) for k in range(1,N+1)])

  # run model and return results
  m.options.RTOL = 1e-8
  m.options.SOLVER = 1  # 1=APOPT, 3=IPOPT
  m.options.IMODE = 1 # Force simulation, DOF = 0 is required
  m.solve(disp=disp)
  print("SOLVER:")
  print(f"Solver status: {m.options.SOLVESTATUS}")
  print(f"Solver time (ms): {m.options.SOLVETIME*1000:.3}")
  print("")
  m.cleanup()

  # Solution summary
  Q_f = sum(values(Q_R[:,0]))
  Q_ret = sum(values(Q_R[:,N]))
  if params['flow_mode'] == 'cross-current':
    Q_perm = sum(values(Q_P[:,1:N+1]))  # special case for cross-current
    Q_perm_comp = [sum(values(Q_P[j,1:N+1])) for j in range(R)]
  if params['flow_mode'] == 'counter-current':
    Q_perm = sum(values(Q_P[:,1]))
    Q_perm_comp = [Q_P[j,1][0] for j in range(R)]

  Q_sw = sum(values(Q_P[:,N+1]))

  print("FLOWS:")
  print(f"Feed flow (mol/s): {Q_f}")
  print(f"Retentate flow (mol/s): {Q_ret}")
  print(f"Permeate flow (mol/s): {Q_perm}")
  print(f"Sweep flow (mol/s): {Q_sw}")
  print(f"Relative flow balance (-): {((Q_f + Q_sw)  - (Q_ret + Q_perm))} ")
  print(f"Stage cut (-): {Q_perm/Q_f}")
  print("")
  print("COMPOSITION:")
  print(f"Components: {params['components']}")
  print(f"Retentate mole fractions: {[Q[0]/Q_ret for Q in Q_R[:,N]]}")
  print(f"Permeate mole fractions: {[Q/Q_perm for Q in Q_perm_comp]}")

  return None


In [243]:
def values(gekko_array):
  # Return values from a gekko array as a list of values
  return [el[0] for el in gekko_array.flatten()]

## Case - Dejaco (2020)
O2/N2 separation with spiral-wound membrane.


In [344]:
# Model parameters
# Data from Dejaco (2020), table 2
params = {
    'type_membrane': 'SW',  # HF = hollow-fiber, SW = spiral-wound
    'flow_mode': 'cross-current', # counter-current, cross-current, co-current
    'OD_fiber': 0, # outer diameter of fiber (m)
    'length_fiber': 0,  # (m)
    'number_fibers': 0, # (-)
    'length_membrane': 0.19,  # (m)
    'height_membrane': 0.394, # (m)
    'components': ["N2", "O2"],
    'q_GPU': [57.5, 145], # component permeance (GPU)
    'section_count': 30,
    'Q_feed':  1.7/3600,  # molar flow rate feed (mol/s)
    'x_feed': [0.793, 0.207], # mole fractions feed (-)
    'Q_sweep': 0.0,  # molar flow rate sweep (mol/s)
    'x_sweep': [0.0, 0.0], # mole fractions sweep (-)
    'p_feed': 274e3,  # (Pa)
    'p_permeate': 101e3 # (Pa)
}

In [345]:
result = membrane_model(params, disp=False)

SOLVER:
Solver status: 1
Solver time (ms): 16.1

FLOWS:
Feed flow (mol/s): 0.00047222222222
Retentate flow (mol/s): 0.000177410612959
Permeate flow (mol/s): 0.00029481160926579994
Sweep flow (mol/s): 0.0
Relative flow balance (-): -4.799925647919201e-15 
Stage cut (-): 0.6243069372716907

COMPOSITION:
Components: ['N2', 'O2']
Retentate mole fractions: [0.8725938923156551, 0.12740610768434496]
Permeate mole fractions: [0.7451022891271281, 0.25489771087287205]


## Case - Scholz (2013)
CO2/propane separation with hollow-fiber membrane.


In [357]:
# Model parameters
# Data from Scholz (2013) Fig 6
params = {
    'type_membrane': 'HF',  # HF = hollow-fiber, SW = spiral-wound
    'flow_mode': 'counter-current', # counter-current, cross-current, co-current
    'OD_fiber': 415e-6, # outer diameter of fiber (m)
    'length_fiber': 0.2,  # (m)
    'number_fibers': 3380, # (-)
    'length_membrane': 0,  # (m)
    'height_membrane': 0, # (m)
    'components': ["CO2", "C3H8"],
    'q_GPU': [203, 0.23], # component permeance (GPU)
    'section_count': 20,
    'Q_feed': 80.4/3600,  # molar flow rate feed (mol/s)
    'x_feed': [0.5, 0.5], # mole fractions feed (-)
    'Q_sweep': 0.0,  # molar flow rate sweep (mol/s)
    'x_sweep': [0.0, 0.0], # mole fractions sweep (-)
    'p_feed': 300e3,  # (Pa)
    'p_permeate': 100e3 # (Pa)
}

In [358]:
membrane_model(params)

SOLVER:
Solver status: 1
Solver time (ms): 14.7

FLOWS:
Feed flow (mol/s): 0.022333333334
Retentate flow (mol/s): 0.0198897190719
Permeate flow (mol/s): 0.002443614261048
Sweep flow (mol/s): 0.0
Relative flow balance (-): 1.052002235324423e-12 
Stage cut (-): 0.10941556392425623

COMPOSITION:
Components: ['CO2', 'C3H8']
Retentate mole fractions: [0.4391151578022605, 0.5608848421977395]
Permeate mole fractions: [0.9955701996339809, 0.004429800366019132]


## Case - Pan (1986) fig 7
H2S/CO2 separation from hydrocarbon mixture with hollow-fiber membrane.


In [239]:
# Model parameters
# Data from Pan (1986) Fig 6
params = {
    'type_membrane': 'HF',  # HF = hollow-fiber, SW = spiral-wound
    'flow_mode': 'counter-current', # counter-current, cross-current, co-current
    'OD_fiber': 200e-6, # outer diameter of fiber (m)
    'length_fiber': 0.36,  # (m)
    'number_fibers': 20, # (-)
    'length_membrane': 0,  # (m)
    'height_membrane': 0, # (m)
    'components': ["H2S", "CO2", "CH4", "N2"],
    'q_GPU': [73.13, 73.13, 3.95, 3.95], # component permeance (GPU)
    'section_count': 30,
    'Q_feed': 10e-3,  # molar flow rate feed (mol/s)
    'x_feed': [0.2209, 0.0338, 0.6605, 0.0837], # mole fractions feed (-)
    'Q_sweep': 0.0,  # molar flow rate sweep (mol/s)
    'x_sweep': [0.0, 0.0, 0.0, 0.0], # mole fractions sweep (-)
    'p_feed': 5245e3,  # (Pa)
    'p_permeate': 92.8e3 # (Pa)
}

In [240]:
membrane_model(params, disp=True)

apm 34.169.61.19_gk_model68 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          256
   Intermediates:           64
   Connections  :           32
   Equations    :          304
   Residuals    :          240
 
 Number of state variables:            240
 Number of total equations: -          240
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 ----------------------------------------------
 Steady State Simulation with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  8.80622E-25  5.00000E-03
    1  1.31447E-26  2.25024E-06
    2  9.28695E-31  2.51393E-08
    